See https://github.com/facebook/Ax/issues/743

In [ ]:
!pip install ax-platform

     |████████████████████████████████| 950 kB 4.1 MB/s 
     |████████████████████████████████| 347 kB 40.2 MB/s 
     |████████████████████████████████| 310 kB 35.7 MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.6.0-py2.py3-none-any.whl size=509889 sha256=4e6dd423b437d3bf045291417889c2c17b1dd76e96d54e6b8ee98de4f8cb763a
  Stored in directory: /root/.cache/pip/wheels/66/b5/89/34c06ad393a6feb72b4cdde46d0f1c667f3e2632960f9df109
Successfully built gpytorch


In [ ]:
# %% imports
import numpy as np
import pandas as pd

from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

unique_components = ["filler_A", "filler_B", "resin_A", "resin_B", "resin_C"]

X_train = np.array([
       [0.4, 0.4, 0. , 0. , 0.2],
       [0.5, 0. , 0. , 0.5, 0. ],
       [0.5, 0.3, 0. , 0.2, 0. ],
       [0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0.6, 0.4, 0. , 0. ],
       [0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0.6, 0.2, 0.2, 0. ]])

X_train = pd.DataFrame(X_train, columns=unique_components)
X_train

,filler_A,filler_B,resin_A,resin_B,resin_C
0,0.4,0.4,0.0,0.0,0.2
1,0.5,0.0,0.0,0.5,0.0
2,0.5,0.3,0.0,0.2,0.0
3,0.5,0.0,0.0,0.5,0.0
4,0.0,0.6,0.4,0.0,0.0
5,0.6,0.0,0.4,0.0,0.0
6,0.0,0.6,0.2,0.2,0.0


In [ ]:
np.random.seed(10)
n_train = X_train.shape[0]
y_train = 100 * np.random.rand(n_train)
y_train

array([77.13206433,  2.07519494, 63.36482349, 74.88038825, 49.85070123,
       22.47966455, 19.80628648])

In [ ]:
# Ax-specific
parameters = [
    {"name": component, "type": "range", "bounds": [0.0, 1.0]}
    for component in unique_components[:-1]
]
parameters

[{'bounds': [0.0, 1.0], 'name': 'filler_A', 'type': 'range'},
 {'bounds': [0.0, 1.0], 'name': 'filler_B', 'type': 'range'},
 {'bounds': [0.0, 1.0], 'name': 'resin_A', 'type': 'range'},
 {'bounds': [0.0, 1.0], 'name': 'resin_B', 'type': 'range'}]

In [ ]:
separator = " + "
composition_constraint = separator.join(unique_components[:-1]) + " <= 1.0"
composition_constraint

'filler_A + filler_B + resin_A + resin_B <= 1.0'

In [ ]:
# skip the pseudo-random suggested points by specifying a custom generation strategy
gs = GenerationStrategy(
    steps=[
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=Models.GPEI,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=3,  # Parallelism limit for this step, often lower than for Sobol
            # More on parallelism vs. required samples in BayesOpt:
            # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
        ),
    ]
)
# setup the experiment
ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="dummy",
    parameters=parameters,
    parameter_constraints=[
        composition_constraint,
    ],
    minimize=True,
)

[INFO 03-30 02:37:22] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-30 02:37:22] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-30 02:37:22] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-30 02:37:22] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter resin_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-30 02:37:2

In [ ]:
# attach the training data
for i in range(n_train):
    ax_client.attach_trial(X_train.iloc[i, :-1].to_dict())
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

[INFO 03-30 02:37:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.4, 'filler_B': 0.4, 'resin_A': 0.0, 'resin_B': 0.0} as trial 0.
[INFO 03-30 02:37:22] ax.service.ax_client: Completed trial 0 with data: {'objective': (77.132064, None)}.
[INFO 03-30 02:37:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} as trial 1.
[INFO 03-30 02:37:22] ax.service.ax_client: Completed trial 1 with data: {'objective': (2.075195, None)}.
[INFO 03-30 02:37:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.3, 'resin_A': 0.0, 'resin_B': 0.2} as trial 2.
[INFO 03-30 02:37:22] ax.service.ax_client: Completed trial 2 with data: {'objective': (63.364823, None)}.
[INFO 03-30 02:37:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} as trial 3.
[INFO 03-30 02:37:22] ax.service.ax_client: Completed tr

In [ ]:
# produce a *single* next suggested experiment, be sure to only run this once
next_experiment, trial_index = ax_client.get_next_trial()
print("next suggested experiment: ", next_experiment)

[INFO 03-30 02:37:29] ax.service.ax_client: Generated new trial 7 with parameters {'filler_A': 0.0, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 1.0}.


next suggested experiment:  {'filler_A': 8.314351981323165e-18, 'filler_B': 0.0, 'resin_A': 2.455264143863484e-17, 'resin_B': 1.0}


In [ ]:
# note that the model fit is poor because of the toy data and randomly generated objective values
# (i.e. this is what we would expect: a bad fit, because the "true" values are nonsense)
best_parameters, metrics = ax_client.get_best_parameters()
print(best_parameters, metrics)

[WARNING 03-30 02:37:29] ax.modelbridge.cross_validation: Metric objective was unable to be reliably fit.
[WARNING 03-30 02:37:29] ax.service.utils.best_point: Model fit is poor; falling back on raw data for best point.
[WARNING 03-30 02:37:29] ax.service.utils.best_point: Model fit is poor and data on objective metric objective is noisy; interpret best points results carefully.


{'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} ({'objective': 2.07519493594015}, {'objective': {'objective': nan}})
